# Document deduplication

Perform a basic deduplication of documents using cosine similarity.


Use **scipy=1.3.1** to avoid library load error `(Library not loaded: @rpath/libopenblas.dylib)`.

In [11]:
import re
import glob
import warnings

from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases, Phraser
from gensim import corpora
from gensim.similarities import Similarity

In [2]:
clean_doc_cache = {}

def cleaned_doc_generator(dir, cleaner, id_pattern=None):
    '''A generator that loads files from a directory and returns a cleaned document.
    This also caches the cleaned data.
    '''

    for fpath in glob.glob(os.path.join(dir, '*.txt')):
        fname = fpath.split('/')[-1]

        if id_pattern is None:
            file_hash = hash(fname)
        else:
            match = re.search(id_pattern, fname)
            if match:
                file_hash = match.group(0)
            else:
                warnings.warn(f'No valid id found in file {fname}. Skipping...')
                continue

        if file_hash not in clean_doc_cache:
            with open(fpath, 'rb') as fl:
                doc = fl.read().decode('utf-8', errors='ignore')
                text = cleaner(doc)
                clean_doc_cache[file_hash] = text

        yield clean_doc_cache[file_hash]

In [ ]:
cleaner = lambda doc: simple_preprocess(doc, deacc=True, max_len=30)

In [8]:
bigram = Phrases(texts, min_count=1)
bigram_phraser = Phraser(bigram)

In [ ]:
dictionary = corpora.Dictionary(texts_bigrams)

In [9]:
w = re.search('\d+', 'hello world')

In [10]:
w.group(0)

AttributeError: 'NoneType' object has no attribute 'group'